In [6]:
import concurrent.futures
import csv
import json
import logging
import os
import random
import sys
from heapq import nlargest

import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoModel, AutoTokenizer

TOP_K = 5
Starting_id=41445
error_list=[]
i = 0
openai.api_type = "azure"
openai.api_base = "https://biocodeeval-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = 'aaccba8e27374383beb397ecdc615ee5' # get this API key from the resource (its not inside the OpenAI deployment portal)

key_bundles = [
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/"),
    ('3a648cbe477c4c0c8061cbdd0a4b8855', "https://biocodeeval-openai2.openai.azure.com/"),
    ('7864e774f3db4066a54c1979672f316c', "https://biocodeeval-openai3.openai.azure.com/")
]

# Contriever part
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

# prompt_construction
def construct_prompt(question, evidences):
    prompt_str = "If the user have browse the following scholar papers with such titles.\n{evidences}\n Please Generate a title for the following abstract of a paper so that the user may be interested: {question}.\Title:"
    evidences_str = "\n".join(evidences)
    prompt_str = prompt_str.replace("{evidences}",evidences_str)
    prompt_str = prompt_str.replace("{question}",question)
    return prompt_str
    
with open('dev_questions.json', 'r', encoding='utf-8') as f:
    jsonObject = json.load(f)
f.close()
input_dict = {}
profile_dict = {}
for item in jsonObject:
    input_dict[item["id"]] = item["input"]
    input_str = input_dict[item["id"]].replace("Generate a title for the following abstract of a paper: ","")
    profile_dict[item["id"]] = [input_str]
    for profile in item["profile"]:
        profile_dict[item["id"]].append("Abstract: "+profile["abstract"]+"\nTitle: " + profile["title"])

with open("naive_prompt_task5_3.tsv", "wb", buffering=0) as out_file:
    with torch.no_grad():
        for indice in profile_dict:
            if float(indice) <= Starting_id:
                continue
            doc_chunks = []
            # Apply tokenizer
            inputs = tokenizer(profile_dict[indice], padding=True, truncation=True, return_tensors='pt')

            # Compute token embeddings
            outputs = model(**inputs)
            embeddings = mean_pooling(outputs[0], inputs['attention_mask'])
            score_index = []
            for i in range(1,len(profile_dict[indice])):
                score_index.append({"ordinal":i,"score":float((embeddings[0] @ embeddings[i]).cpu().detach())})
            top_score_index = nlargest(TOP_K, score_index, key=lambda item: item["score"])
            top_score_docs = [profile_dict[indice][i] for i in (i["ordinal"] for i in top_score_index)]
            prompt = construct_prompt(profile_dict[indice][0],top_score_docs)
            my_dict = {}
            my_dict["role"] = "user"
            my_dict["content"] = prompt
            l = []
            l.append(my_dict)
            try:
                key_bundle = key_bundles[i%3]
                openai.api_key, openai.api_base = key_bundle
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                write_str = bytes(indice+"\t"+result+"\n", 'utf-8')
                out_file.write(write_str)
                print(result)
                i = i + 1
            except openai.error.Timeout:
                print("Timeout",indice)
                key_bundle = key_bundles[i%3]
                openai.api_key, openai.api_base = key_bundle
                try:
                    response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                    result = response.choices[0].message["content"].replace('\n', ' ')
                    write_str = bytes(indice+"\t"+result+"\n", 'utf-8')
                    out_file.write(write_str)
                    i = i + 1
                except openai.error.Timeout:
                    print("Timeout",indice)
                    key_bundle = key_bundles[i%3]
                    openai.api_key, openai.api_base = key_bundle
                    response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                    result = response.choices[0].message["content"].replace('\n', ' ')
                    write_str = bytes(indice+"\t"+result+"\n", 'utf-8')
                    out_file.write(write_str)
                except Exception as e:
                    print(f"An error occurred: {str(e)}")
                    error_list.append(i)
                #print(result)
            except openai.error.InvalidRequestError:
                print("InvalidRequestError",indice)
                error_list.append(i)
                #print(result)
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
            i = i + 1

The Power of Symmetry in Object Recognition: A Computational Framework for Recovering and Grouping Parts in Cluttered Scenes.
Improved Range Image Registration through Segmentation-Based Correspondence Matching
Efficient Exploitation of Linear Channel Codes for Asymmetric Lossless Distributed Source Coding Using Syndrome Formers and Inverse Syndrome Formers
Efficient and Secure Password-Authenticated Group Diffie-Hellman Key Exchange for N-Party Setting
Improving the Gallant-Lambert-Vanstone Method for Speeding Up Scalar Multiplication on Elliptic Curves
Advancements in 3D Mesh Data: Novel Techniques for Reliable Surface Registration of Non-Rigid and Articulated Objects.
"Enhancing Perceptual Accuracy in Dynamic Textured Sequences through Temporal Modeling and Maximum Entropy Spectral Analysis"
Perspective Skewed Mirror Symmetry and Invariance Research
Quantum-Inspired Particle Swarm Optimization for String Pattern Recognition with Evolving Spiking Neural Networks
Exploring the Richnes

In [2]:
# TASK 4-7 


import csv
import json
import pandas as pd
import re
class LazyDecoder(json.JSONDecoder):
    def decode(self, s, **kwargs):
        regex_replacements = [
            (re.compile(r'([^\\])\\([^\\])'), r'\1\\\\\2'),
            (re.compile(r',(\s*])'), r'\1'),
        ]
        for regex, replacement in regex_replacements:
            s = regex.sub(replacement, s)
        return super().decode(s, **kwargs)
    
task_num = "4"
prompt_version = "1.67"
example_num = "30"

id2result = {}
id2input = {}

# with open('input/task'+task_num+'_dev_questions.json', 'r', encoding='utf-8') as f:
#     inputs = json.load(f)
# for i in inputs:
#     id2input[i["id"]]=i["input"].replace("Paraphrase the following tweet without any explanation before or after it: ","")

# with open('task'+task_num+'_dev_outputs.json', 'r', encoding='utf-8') as f:
#     labels = json.load(f)
results = {
    "task": "LaMP_"+task_num,
    "golds":[]
}
result_df = pd.read_csv('cot_prompt_agg_reasoning_task5_5_examples_prompt_random_v1.67_test.tsv', sep='\t',header=None, names=["id","output","prompt"])
for i in range(len(result_df["id"])):
    output_result = result_df["output"][i]
    results["golds"].append({"id":str(result_df["id"][i]),"output":output_result.replace("\"","").replace(",","")})

with open('task5_result_user_based.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=4)
with open('task5_result_user_based.json', 'r') as f:
    s1 = json.load(f, cls=LazyDecoder)

len(results["golds"])
# with open('task'+task_num+'_dev_outputs_gold.json', 'w', encoding='utf-8') as f:
#     json.dump(removed_labels, f, indent=4)
# with open('task'+task_num+'_dev_outputs_gold.json', 'r', encoding='utf-8') as f:
#     s1 = json.load(f)

    
# with open('review_data/review_task'+task_num+'_CoT_promptv'+prompt_version+'_'+example_num+'_examples_mixed.json', 'w', encoding='utf-8') as f:
#     json.dump(results_for_review, f, indent=4)

2498